**Descrição**

Mineração de dados da Covid-19 para cruzar com os dados de receitas dos municipios do RN e investigar possível impacto nas receitas previstas e/ou arrecadadas declaradas pelos munícipios bimestralmente. 

O notebook recebe um csv buscado do banco do TCE-RN contendo as colunas:
  - NomeOrgao	
  - Ano	
  - Bimestre	
  - ValorPrevistoInicial	
  - ValorRealizadoBimestre

O notebook adiciona a este arquivo uma coluna "Casos" sendo nulo para todos os outros anos sem a Covid-19 e adicionando os valores de casos registrados pelo estado.

O registro dos casos é feito de forma diária e com margem de erro considerável, pois o boletim epidemiológico oficial ([sesapRN](http://www.saude.rn.gov.br/Conteudo.asp?TRAN=ITEM&TARG=239115&ACT=&PAGE=0&PARM=&LBL=ACERVO+DE+MAT%C9RIAS)) não explica a qualidade e precisão dos dados, contendo amostras sujas e dados faltosos. 

In [ ]:
!pip uninstall pandas-profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install Unidecode

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
import plotly.express as px
import re
import unidecode

In [ ]:
df = pd.read_excel('DOC000000000233213.XLSX')
df.columns = df.columns.str.replace(' ','_')
df.DATA_NOT = pd.to_datetime(df.DATA_NOT, errors='coerce')
print(len(df))

64585


In [ ]:
profile = ProfileReport(df, title='Profiling',html={'style':{'full_width':True}})
profile.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df['mes'] = pd.DatetimeIndex(df['DATA_NOT']).month

df = df[df['mes'].notna()]
df['mes'] = df['mes'].astype(int)
df['bimestre'] = (df['mes']/2+0.9).astype(int)
del df['mes']

In [ ]:
df = df[df['UF'] == 'RN']
df['MUN_RES'] = df['MUN_RES'].str.upper()
df['MUN_RES'] = df['MUN_RES'].apply(lambda x: unidecode.unidecode(x))
df = df[df['RESULTADO'] == 'Positivo']
groups = df.groupby(['MUN_RES','bimestre'])['RESULTADO'].count().reset_index(name="Casos")
groups2 = groups[groups['bimestre'] == 2]
groups2.head()

,MUN_RES,bimestre,Casos
0,ACARI,2,1
2,ACU,2,41
4,AFONSO BEZERRA,2,1
6,ALEXANDRIA,2,4
9,ALTO DO RODRIGUES,2,2


In [ ]:
casos={}
for index, row in groups2.iterrows():
  casos[row['MUN_RES']] = row['Casos']

In [ ]:
df2 = pd.read_excel('ListaFinal.xls')
df2['NomeOrgao'] = df2['NomeOrgao'].apply(lambda x: unidecode.unidecode(x))
df3 = df2[df2['Ano'] == 2020]
df3 = df3[df3['Bimestre'] > 1]

In [ ]:
to_append = []
for index, row in df3.iterrows():
    x = re.sub('PREFEITURA MUNICIPAL DE ', '', row['NomeOrgao'])
    if str(x) in casos.keys():
      to_append.append(casos[x])
    else:
      to_append.append(0)
df3['Casos'] = to_append
df3

,NomeOrgao,Ano,Bimestre,ValorPrevistoInicial,ValorRealizadoBimestre,Casos
5,PREFEITURA MUNICIPAL DE ACARI,2020,2,41726856.00,4977448.45,1
11,PREFEITURA MUNICIPAL DE ALEXANDRIA,2020,2,63091108.86,9736458.49,4
17,PREFEITURA MUNICIPAL DE ALMINO AFONSO,2020,2,35889150.00,2874852.58,0
23,PREFEITURA MUNICIPAL DE ANGICOS,2020,2,48952221.00,4474287.76,0
29,PREFEITURA MUNICIPAL DE APODI,2020,2,98683750.00,13444698.88,33
...,...,...,...,...,...,...
688,PREFEITURA MUNICIPAL DE VENHA VER,2020,2,32063179.50,2841795.26,0
694,PREFEITURA MUNICIPAL DE VERA CRUZ,2020,2,38198584.00,5509333.43,0
700,PREFEITURA MUNICIPAL DE VILA FLOR,2020,2,23782806.58,2784010.21,0
706,PREFEITURA MUNICIPAL DE VARZEA,2020,2,29170000.00,3279117.08,0


In [ ]:
df2['Casos'] = df3['Casos']
df2['Casos'].fillna(0, inplace=True)
df2['Casos'] = df2['Casos'].astype(int)

In [ ]:
df2.query("NomeOrgao == 'PREFEITURA MUNICIPAL DE NATAL'")

,NomeOrgao,Ano,Bimestre,ValorPrevistoInicial,ValorRealizadoBimestre,Casos
354,PREFEITURA MUNICIPAL DE NATAL,2018,1,2.882936e+09,4.644974e+08,0
355,PREFEITURA MUNICIPAL DE NATAL,2018,2,2.882936e+09,3.596798e+08,0
356,PREFEITURA MUNICIPAL DE NATAL,2019,1,3.223750e+09,5.204203e+08,0
357,PREFEITURA MUNICIPAL DE NATAL,2019,2,3.223750e+09,4.221176e+08,0
358,PREFEITURA MUNICIPAL DE NATAL,2020,1,3.530050e+09,5.674892e+08,0
359,PREFEITURA MUNICIPAL DE NATAL,2020,2,3.530050e+09,4.465777e+08,681


In [ ]:
df2.to_csv('receitasxcovid.csv')